# Building a Decision Tree Model For Anova Insurance (Google Colab Notebook)

This notebook follows a **step-by-step, end-to-end workflow** to build an **interpretable Decision Tree classifier** that predicts applicant health status:
- **0 = Healthy**
- **1 = Unhealthy**

It also exports final outputs for download:
1. **Predictions CSV**
2. **Excel workbook (2 sheets/tabs)**:
   - `1_Decision Tree Model For Anova Insurance`
   - `2_Decision Tree Success Criteria`

---


In [ ]:
# ==============================
# Step 0: Setup (Install/Import)
# ==============================
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve
)

# Colab-only: file upload and download helpers
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False

print("✅ Imports complete. IN_COLAB =", IN_COLAB)

In [ ]:
# ======================================================
# Step 5.1: Data Ingestion (Upload CSV/XLSX in Colab)
# ======================================================
# You will be prompted to upload your dataset file.
# Supported: .csv, .xlsx

if IN_COLAB:
    uploaded = files.upload()  # choose your file
    uploaded_filename = list(uploaded.keys())[0]
    print("Uploaded file:", uploaded_filename)
else:
    # If not in Colab, set your local path here:
    uploaded_filename = "YOUR_DATA_FILE.csv"
    print("Running outside Colab. Using:", uploaded_filename)

def load_dataset(path: str) -> pd.DataFrame:
    if path.lower().endswith(".csv"):
        return pd.read_csv(path)
    if path.lower().endswith(".xlsx") or path.lower().endswith(".xls"):
        # Read the first sheet by default
        return pd.read_excel(path)
    raise ValueError("Unsupported file type. Please upload a .csv or .xlsx file.")

df = load_dataset(uploaded_filename)

print("✅ Loaded dataset.")
print("Shape:", df.shape)
df.head()

In [ ]:
# =========================================
# Step 5.1 (cont.): Basic Data Validation
# =========================================
required_target = "Target"
if required_target not in df.columns:
    raise ValueError(f"Missing required target column: '{required_target}'. Found columns: {list(df.columns)}")

# Basic checks
print("Target value counts:\n", df["Target"].value_counts(dropna=False))

# Ensure target is binary {0,1}
unique_targets = set(pd.Series(df["Target"]).dropna().unique().tolist())
if not unique_targets.issubset({0, 1}):
    raise ValueError(f"Target must be binary (0/1). Found: {unique_targets}")

# Show dtypes summary
display(df.dtypes.value_counts())
df.info()

## Step 5.2: Exploratory Data Analysis (EDA)

This section covers:
- Target distribution (class imbalance)
- Summary stats for numeric columns (outliers/odd ranges)
- Missing value % per column + missingness visualization
- Correlation checks (numeric vs target)
- Category distributions for ordinal (0/1/2) + nominal features


In [ ]:
# =========================
# Step 5.2: EDA
# =========================

# 1) Target distribution
plt.figure()
df["Target"].value_counts().sort_index().plot(kind="bar")
plt.title("Target Distribution (0=Healthy, 1=Unhealthy)")
plt.xlabel("Target")
plt.ylabel("Count")
plt.show()

# 2) Numeric summary stats
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", numeric_cols)
display(df[numeric_cols].describe().T)

# 3) Missing values % per column
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(missing_pct.to_frame("missing_%"))

# Missingness "heatmap" (matplotlib imshow)
plt.figure(figsize=(12, 4))
plt.imshow(df.isna().T, aspect="auto", interpolation="nearest")
plt.title("Missing Values Map (rows vs columns)")
plt.yticks(range(len(df.columns)), df.columns, fontsize=8)
plt.xlabel("Row index")
plt.show()

# 4) Correlation with Target (numeric only)
if "Target" in numeric_cols:
    corr = df[numeric_cols].corr(numeric_only=True)["Target"].sort_values(ascending=False)
    display(corr.to_frame("corr_with_target"))

    # Correlation matrix plot for numeric columns
    corr_matrix = df[numeric_cols].corr(numeric_only=True)
    plt.figure(figsize=(10, 8))
    plt.imshow(corr_matrix, aspect="auto")
    plt.title("Correlation Matrix (Numeric Features)")
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=8)
    plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=8)
    plt.colorbar()
    plt.tight_layout()
    plt.show()

# 5) Category distributions (best-effort detection)
# Expected ordinal features (0/1/2) — adjust if your dataset uses different names
expected_ordinal = ["Smoking", "Alcohol", "Diet", "MentalHealth", "PhysicalActivity", "MedicalHistory", "Allergies"]
present_ordinal = [c for c in expected_ordinal if c in df.columns]

for c in present_ordinal:
    plt.figure()
    df[c].value_counts(dropna=False).sort_index().plot(kind="bar")
    plt.title(f"Distribution: {c}")
    plt.xlabel(c)
    plt.ylabel("Count")
    plt.show()

# Nominal features (expected)
expected_nominal = ["Diet_Type", "Blood_Group"]
present_nominal = [c for c in expected_nominal if c in df.columns]

for c in present_nominal:
    plt.figure()
    df[c].value_counts(dropna=False).plot(kind="bar")
    plt.title(f"Distribution: {c}")
    plt.xlabel(c)
    plt.ylabel("Count")
    plt.show()

print("✅ EDA complete.")

## Step 5.3: Data Cleaning

Implements:
- Convert **negative Age** to `NaN` (then impute)
- Basic sanity checks for impossible values (BMI <= 0, Sleep_Hours out of range, etc.)
- Missing value handling via imputers in the modeling pipeline


In [ ]:
# =========================
# Step 5.3: Data Cleaning
# =========================

df_clean = df.copy()

# Convert negative Age to NaN (safe default)
if "Age" in df_clean.columns:
    df_clean.loc[df_clean["Age"] < 0, "Age"] = np.nan

# Remove / correct impossible values (safe defaults)
# You can extend these rules based on domain knowledge.
if "BMI" in df_clean.columns:
    df_clean.loc[df_clean["BMI"] <= 0, "BMI"] = np.nan

if "Sleep_Hours" in df_clean.columns:
    df_clean.loc[(df_clean["Sleep_Hours"] < 0) | (df_clean["Sleep_Hours"] > 24), "Sleep_Hours"] = np.nan

if "Exercise_Hours" in df_clean.columns:
    df_clean.loc[(df_clean["Exercise_Hours"] < 0) | (df_clean["Exercise_Hours"] > 24), "Exercise_Hours"] = np.nan

if "Water_Intake" in df_clean.columns:
    df_clean.loc[(df_clean["Water_Intake"] < 0) | (df_clean["Water_Intake"] > 20), "Water_Intake"] = np.nan

print("✅ Cleaning rules applied (Age negatives, impossible ranges set to NaN).")
print("Shape:", df_clean.shape)
df_clean.head()

## Step 5.4: Feature Engineering + Encoding

Optional engineered features:
- BMI category
- Blood pressure category
- Glucose category
- Lifestyle risk score

Encoding strategy:
- Ordinal 0/1/2 features → keep as numeric (order preserved)
- Nominal features (Diet_Type, Blood_Group) → One-Hot Encoding


In [ ]:
# ======================================
# Step 5.4: Feature Engineering (Optional)
# ======================================
df_fe = df_clean.copy()

def bmi_category(bmi):
    if pd.isna(bmi): return np.nan
    if bmi < 18.5: return "Underweight"
    if bmi < 25: return "Normal"
    if bmi < 30: return "Overweight"
    return "Obese"

def bp_category(sys_bp):
    # Using systolic BP categories (simplified)
    if pd.isna(sys_bp): return np.nan
    if sys_bp < 120: return "Normal"
    if sys_bp < 130: return "Elevated"
    if sys_bp < 140: return "High_Stage1"
    return "High_Stage2"

def glucose_category(glu):
    # Simplified fasting glucose buckets
    if pd.isna(glu): return np.nan
    if glu < 100: return "Normal"
    if glu < 126: return "Prediabetic"
    return "DiabeticRange"

# Create engineered features if base columns exist
if "BMI" in df_fe.columns:
    df_fe["BMI_Category"] = df_fe["BMI"].apply(bmi_category)

if "Blood_Pressure" in df_fe.columns:
    df_fe["BP_Category"] = df_fe["Blood_Pressure"].apply(bp_category)

if "Glucose_Level" in df_fe.columns:
    df_fe["Glucose_Category"] = df_fe["Glucose_Level"].apply(glucose_category)

# Lifestyle risk score (simple additive index) if columns exist
lifestyle_cols = [c for c in ["Smoking", "Alcohol", "Diet", "PhysicalActivity"] if c in df_fe.columns]
if lifestyle_cols:
    # Higher smoking/alcohol increases risk, higher diet/physical activity might reduce risk.
    # We'll invert Diet and PhysicalActivity to reflect risk (0 poor -> higher risk; 2 good -> lower risk).
    risk_score = pd.Series(0, index=df_fe.index, dtype=float)
    if "Smoking" in df_fe.columns: risk_score += df_fe["Smoking"].fillna(0)
    if "Alcohol" in df_fe.columns: risk_score += df_fe["Alcohol"].fillna(0)
    if "Diet" in df_fe.columns: risk_score += (2 - df_fe["Diet"].fillna(1))  # default 1
    if "PhysicalActivity" in df_fe.columns: risk_score += (2 - df_fe["PhysicalActivity"].fillna(1))
    df_fe["Lifestyle_Risk_Score"] = risk_score

print("✅ Feature Engineering complete.")
df_fe.head()

## Step 5.5: Train/Test Split

- Uses stratification to preserve class distribution.


In [ ]:
# =========================
# Step 5.5: Train/Test Split
# =========================

X = df_fe.drop(columns=["Target"])
y = df_fe["Target"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Train target distribution:\n", y_train.value_counts(normalize=True))
print("Test target distribution:\n", y_test.value_counts(normalize=True))

## Step 5.6: Baseline Decision Tree Model

Build a baseline Decision Tree using a preprocessing pipeline:
- Numerical → median imputation
- Ordinal (0/1/2) → most frequent imputation (kept numeric)
- Nominal → most frequent + one-hot encoding


In [ ]:
# =====================================
# Step 5.6: Baseline Decision Tree Model
# =====================================

# Identify column types
expected_ordinal = ["Smoking", "Alcohol", "Diet", "MentalHealth", "PhysicalActivity", "MedicalHistory", "Allergies"]
ordinal_cols = [c for c in expected_ordinal if c in X.columns]

expected_nominal = ["Diet_Type", "Blood_Group", "BMI_Category", "BP_Category", "Glucose_Category"]
nominal_cols = [c for c in expected_nominal if c in X.columns and X[c].dtype == "object"]

# Numeric columns = number dtype excluding ordinal_cols
numeric_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c]) and c not in ordinal_cols]

print("Numeric:", numeric_cols)
print("Ordinal (0/1/2):", ordinal_cols)
print("Nominal:", nominal_cols)

# Preprocessors
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

ordinal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent"))
    # No scaling; keep order
])

nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("ord", ordinal_transformer, ordinal_cols),
        ("nom", nominal_transformer, nominal_cols),
    ],
    remainder="drop"
)

baseline_model = DecisionTreeClassifier(random_state=42)

baseline_pipe = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", baseline_model)
])

baseline_pipe.fit(X_train, y_train)

print("✅ Baseline model trained.")

## Step 5.7: Hyperparameter Tuning (Control Overfitting)

Uses `GridSearchCV` with cross-validation.  
Optimizes primarily for **Recall (Unhealthy = 1)** (insurance risk sensitivity).


In [ ]:
# =====================================
# Step 5.7: Hyperparameter Tuning
# =====================================

param_grid = {
    "model__max_depth": [3, 4, 5, 6, 8, 10, None],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__max_features": [None, "sqrt", "log2"],
    "model__class_weight": [None, "balanced"]
}

grid = GridSearchCV(
    estimator=baseline_pipe,
    param_grid=param_grid,
    scoring="recall",  # recall for positive class (1) by default in sklearn when labels are {0,1}
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

best_pipe = grid.best_estimator_
print("✅ Best params:", grid.best_params_)
print("Best CV Recall:", grid.best_score_)

## Step 5.8: Model Evaluation (Technical + Insurance Lens)

Technical metrics:
- Confusion Matrix
- Accuracy
- Precision (Unhealthy)
- Recall (Unhealthy) **(priority)**
- F1-score
- ROC-AUC

Insurance lens:
- Explicitly count **False Negatives** (Unhealthy predicted as Healthy)


In [ ]:
# =========================
# Step 5.8: Evaluation
# =========================

y_pred = best_pipe.predict(X_test)

# If model supports probabilities
y_proba = None
if hasattr(best_pipe.named_steps["model"], "predict_proba"):
    y_proba = best_pipe.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
rec = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)

print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}  (Unhealthy=1)")
print(f"Recall   : {rec:.4f}  (Unhealthy=1)  ✅ priority")
print(f"F1-score : {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Healthy(0)", "Unhealthy(1)"])
disp.plot(values_format="d")
plt.title("Confusion Matrix")
plt.show()

tn, fp, fn, tp = cm.ravel()
print("Insurance Lens:")
print("False Negatives (FN) = Unhealthy predicted as Healthy =", fn, "❗ highest financial risk")

# ROC-AUC
if y_proba is not None:
    auc = roc_auc_score(y_test, y_proba)
    print(f"ROC-AUC  : {auc:.4f}")
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], linestyle="--")
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.show()
else:
    print("ROC-AUC skipped (predict_proba not available).")

## Step 5.9: Interpretability Outputs

- Feature importance ranking
- Visualize tree (limited depth for business readability)
- Extract a few simple rules (best-effort)


In [ ]:
# =====================================
# Step 5.9: Interpretability Outputs
# =====================================

# Get feature names after preprocessing
pre = best_pipe.named_steps["preprocess"]
model = best_pipe.named_steps["model"]

feature_names = []

# Numeric
feature_names += numeric_cols

# Ordinal
feature_names += ordinal_cols

# Nominal (one-hot)
if nominal_cols:
    ohe = pre.named_transformers_["nom"].named_steps["onehot"]
    ohe_names = ohe.get_feature_names_out(nominal_cols).tolist()
    feature_names += ohe_names

# Feature importance
importances = model.feature_importances_
fi = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False)
display(fi.head(20))

plt.figure(figsize=(10, 5))
plt.bar(range(min(20, len(fi))), fi["importance"].values[:20])
plt.xticks(range(min(20, len(fi))), fi["feature"].values[:20], rotation=90, fontsize=8)
plt.title("Top 20 Feature Importances (Decision Tree)")
plt.tight_layout()
plt.show()

# Plot tree (limit depth for readability)
plt.figure(figsize=(18, 10))
plot_tree(
    model,
    feature_names=feature_names,
    class_names=["Healthy(0)", "Unhealthy(1)"],
    filled=True,
    max_depth=3,
    fontsize=8
)
plt.title("Decision Tree (Top Levels, max_depth=3 view)")
plt.show()

print("✅ Interpretability outputs generated.")

## Step 5.10: Risk Bands + Premium Mapping Logic

Probability-based risk bands (recommended):
- **Low Risk:** p(Unhealthy) < 0.30
- **Medium Risk:** 0.30–0.60
- **High Risk:** > 0.60

Premium action mapping (example):
- Low → Standard / Discount eligible
- Medium → Standard + small loading / caution
- High → Higher premium / medical review / conditional coverage


In [ ]:
# ============================================
# Step 5.10: Risk Bands + Premium Mapping Logic
# ============================================

def risk_band(p_unhealthy: float) -> str:
    if pd.isna(p_unhealthy):
        return "Unknown"
    if p_unhealthy < 0.30:
        return "Low"
    if p_unhealthy <= 0.60:
        return "Medium"
    return "High"

def premium_action(band: str) -> str:
    mapping = {
        "Low": "Standard / Discount eligible",
        "Medium": "Standard + small loading / caution",
        "High": "Higher premium / medical review / conditional coverage",
        "Unknown": "Manual review required"
    }
    return mapping.get(band, "Manual review required")

# Create scored test set output table
output = X_test.copy()
output["Actual_Target"] = y_test.values
output["Predicted_Target"] = y_pred

if y_proba is not None:
    output["P_Unhealthy"] = y_proba
    output["Risk_Band"] = output["P_Unhealthy"].apply(risk_band)
else:
    # Fallback: simple bands based on predicted class
    output["P_Unhealthy"] = np.nan
    output["Risk_Band"] = np.where(output["Predicted_Target"] == 1, "High", "Low")

output["Premium_Action"] = output["Risk_Band"].apply(premium_action)

display(output.head(10))
print("✅ Risk banding and premium mapping complete.")

## Step 5.11: Business Validation Checklist (Guidance)

Use this checklist with underwriting stakeholders:
- Do the top features and splits make medical sense?
- Are there unfair/non-actionable patterns?
- Is the tree too deep/complex for explanation?
- Are False Negatives within acceptable limits?


## Step 5.12: Final Packaging + Export Outputs (Downloadable)

This step exports:
1. `Anova_DecisionTree_Predictions.csv` (test predictions + risk band + premium action)
2. `Anova_DecisionTree_Outputs.xlsx` with **two tabs**:
   - `1_Decision Tree Model For Anova Insurance` (predictions + key metrics)
   - `2_Decision Tree Success Criteria` (technical + business success criteria)

If running in Colab, it triggers downloads automatically.


In [ ]:
# =====================================================
# Export outputs for download (CSV + Excel with 2 tabs)
# =====================================================

# 1) Save predictions CSV
pred_csv_path = "Anova_DecisionTree_Predictions.csv"
output.to_csv(pred_csv_path, index=False)
print("✅ Saved:", pred_csv_path)

# 2) Build Success Criteria sheet (Step 6)
success_criteria = pd.DataFrame({
    "Category": [
        "Technical", "Technical", "Technical", "Technical", "Technical",
        "Business", "Business", "Business"
    ],
    "Success_Criteria": [
        "Accuracy (baseline indicator)",
        "Recall for Unhealthy (primary)",
        "Precision + F1-score (balanced performance)",
        "ROC-AUC (separation capability)",
        "Low False Negatives (unhealthy predicted healthy)",
        "Better premium differentiation",
        "Reduced underwriting risk",
        "Scalable + explainable health scoring"
    ],
    "How_to_Measure": [
        "accuracy_score on test set",
        "recall_score(pos_label=1) on test set",
        "precision_score & f1_score(pos_label=1) on test set",
        "roc_auc_score using predicted probabilities",
        "Confusion matrix FN count should be minimized",
        "Premium tiers align with risk bands and outcomes",
        "Lower claim losses from underpriced high-risk policies (business KPI)",
        "Stakeholder adoption + auditability of tree rules"
    ]
})

# 3) Create a metrics summary to include in Sheet 1
metrics_summary = pd.DataFrame({
    "Metric": ["Accuracy", "Precision (Unhealthy=1)", "Recall (Unhealthy=1)", "F1 (Unhealthy=1)", "False Negatives (FN)", "True Positives (TP)", "False Positives (FP)", "True Negatives (TN)"],
    "Value": [acc, prec, rec, f1, int(fn), int(tp), int(fp), int(tn)]
})

if y_proba is not None:
    metrics_summary = pd.concat([metrics_summary, pd.DataFrame({"Metric":["ROC-AUC"], "Value":[auc]})], ignore_index=True)

# 4) Export Excel with two tabs
xlsx_path = "Anova_DecisionTree_Outputs.xlsx"
sheet1_name = "1_Decision Tree Model For Anova Insurance"
sheet2_name = "2_Decision Tree Success Criteria"

with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
    # Sheet 1: metrics + predictions
    metrics_summary.to_excel(writer, sheet_name=sheet1_name, index=False, startrow=0)
    # Leave a gap, then write predictions under metrics
    startrow = len(metrics_summary) + 3
    output.to_excel(writer, sheet_name=sheet1_name, index=False, startrow=startrow)

    # Sheet 2: success criteria table
    success_criteria.to_excel(writer, sheet_name=sheet2_name, index=False)

print("✅ Saved:", xlsx_path)

# 5) Download files (Colab)
if IN_COLAB:
    files.download(pred_csv_path)
    files.download(xlsx_path)
else:
    print("Run in Colab to auto-download, or find files in your working directory.")

## Step 1 (last item): Deployment Readiness + Monitoring (Guidance)

When you productionize:
- Save the trained pipeline (preprocessor + model)
- Version dataset + model parameters
- Monitor:
  - data drift (feature distributions)
  - performance drift (recall for Unhealthy)
  - FN rate over time (highest insurer risk)
